In [4]:
import pandas as pd

# ---------- Pipeline para transformar los audios en los ceptogramas log-mel ------------- #


# ----------- Cargar datasets --------------#

train_noisy_df = pd.read_csv('../taa-2025-freesound-audio-tagging/train_noisy.csv')
train_curated_df = pd.read_csv('../taa-2025-freesound-audio-tagging/train_curated.csv')
vocabulary_df = pd.read_csv('../taa-2025-freesound-audio-tagging/vocabulary.csv')


In [5]:
train_curated_df.describe()
train_curated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4970 entries, 0 to 4969
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fname   4970 non-null   object
 1   labels  4970 non-null   object
dtypes: object(2)
memory usage: 77.8+ KB


In [ ]:
'../tt/train/'+train_df[algo]

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib

In [ ]:
# Load CSV with filenames and labels
train_curated_df = pd.read_csv('../taa-2025-freesound-audio-tagging/train_curated.csv')

# Build lists of file paths and labels
file_paths = train_curated_df['fname'].apply(lambda x: str(pathlib.Path('../taa-2025-freesound-audio-tagging/train_curated') / x)).tolist()
labels = train_curated_df['labels'].tolist()  # If multiple labels, you may need to split and process

# If labels are single class per file, encode as integers:
unique_labels = sorted(set(l for label in labels for l in label.split(',')))
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
labels_idx = [label_to_index[label] if ',' not in label else label_to_index[label.split(',')[0]] for label in labels]

# Create tf.data.Dataset
file_paths_ds = tf.data.Dataset.from_tensor_slices(file_paths)
labels_ds = tf.data.Dataset.from_tensor_slices(labels_idx)
dataset = tf.data.Dataset.zip((file_paths_ds, labels_ds))

def load_wav(file_path, label):
    audio = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(audio, desired_channels=1, desired_samples=16000)
    audio = tf.squeeze(audio, axis=-1)
    return audio, label

dataset = dataset.map(load_wav, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

print('Example batch:')
for audio, label in dataset.take(1):
    print(audio.shape, label.shape)
    break